In [ ]:
## packages
import intake
import time
import xarray as xr
from distributed import LocalCluster, Client

In [ ]:
# If not using NCAR HPC, use the LocalCluster
cluster = LocalCluster(n_workers=16)
# cluster.scale(32)

client = Client(cluster)
client

In [ ]:
cluster

In [ ]:
## get catalog of available data
catalog = intake.open_esm_datastore(
    "https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json"
)

## subset for temperature data
catalog_subset = catalog.search(
    variable="TREFHT", frequency="monthly", experiment="historical"
)

## open_dataset kwargs
xr_kwargs = dict(
    engine="zarr",
    decode_timedelta=True,
    # chunks=dict(member=1, time=1024, nlat=None, nlon=None),
)

## kwargs for opening data
kwargs = dict(
    aggregate=True,
    xarray_open_kwargs=xr_kwargs,
    zarr_kwargs={"consolidated": True},
    storage_options={"anon": True},
)

## load data
dsets = catalog_subset.to_dataset_dict(**kwargs)
data = dsets["atm.historical.monthly.cmip6"]

In [ ]:
data["TREFHT"]

In [ ]:
## specify lat/lon range
lonlat_vals = dict(lon=slice(285, 295), lat=slice(35, 45))

## trim in lon/lat space
data_ = data["TREFHT"].sel(lonlat_vals).isel(member_id=slice(None,16))

In [ ]:
t0 = time.time()
data_.load();
t1 = time.time()

In [ ]:
print(f"{t1-t0:.1f} s")

In [ ]:
client.shutdown()

In [ ]:
data_

In [ ]:
print(f"{t1-t0:.1f} s")